In [2]:
# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver

# selenium으로 키를 조작하기 위한 import
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time

# Open browser

In [57]:
# 크롬드라이버 실행. 4.6 버전부터 크롬드라이버 별도 다운로드 및 경로 설정 불필요
driver = webdriver.Chrome() 

#크롬 드라이버에 url 주소 넣고 실행
site = 'https://www.vivino.com/casillero-del-diablo-cabernet-sauvignon-reserva-central-valley/w/1135067?ref=nav-search#all_reviews'
driver.get(site)

# 페이지가 완전히 로딩되도록 3초동안 기다림
time.sleep(3)

# Get community review list

In [58]:
# Open community review window by clicking Show more reviews
path = "//button[@tabindex='0']"
#path = "//span[text()='Show more reviews']" # successful
#path = '//*[@id="all_reviews"]/div[3]/div[1]/button' # NoSuchElementException

driver.find_element(By.XPATH, path).click()
time.sleep(1)

# Show reviews by recent
path = '//*[@data-testid="baseModalBackdrop"]/div[2]/div[2]/div[2]/div/a[1]'

driver.find_element(By.XPATH, path).click()
time.sleep(1)

# Get reviews

In [59]:
class_review = "communityReview__reviewText--2bfLj"
class_date = "anchor_anchor__m8Qi- reviewAnchor__anchor--2NKFw reviewDate__reviewDate--49vpM undefined"
class_date = class_date.replace(' ', '.')
   

max_rev = 100
max_scr = 5
time_scr = 2

def extract_text(class_name, driver=driver, by=By.CLASS_NAME):
    l = driver.find_elements(by, class_name)
    return [x.text for x in l]


reviews = list()
dates = list()
n_scr = 0

while True:
    list_r = extract_text(class_review)
    list_d = extract_text(class_date)

    n = len(list_r) - len(reviews)
    if n > 0: # get new n reviews
        reviews.extend(list_r[-n:])
        dates.extend(list_d[-n:])
        n_scr = 0
    else:
        n_scr += 1

    if ((len(reviews) > max_rev) or (n_scr > max_scr)):
        break
    else:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(time_scr)

#len(reviews), len(dates)

# close browser
driver.close()

(113, 113)

# Save reviews

In [83]:
import pandas as pd

df_reviews = pd.DataFrame.from_dict({'date':dates, 'review':reviews})
df_reviews['date'] = pd.to_datetime(df_reviews['date'])
df_reviews.head()

,date,review
0,2024-01-20,"En liten skarp knekk i smaken. Ok fredagsvin,m..."
1,2024-01-19,Aight
2,2024-01-18,무난한 맛 가성비 좋은듯
3,2024-01-17,oak cherry black cherry chocolate blackcurrant...
4,2024-01-17,"Muito bom, combina muito bem com carnes vermel..."


In [84]:
f = 'wine_reviews'
df_reviews.to_csv(f, index=False)  